# Neo4j con Python

## py2neo

py2neo is one of Neo4j's Python drivers. It offers a fully-featured interface for interacting with your data in Neo4j.

https://py2neo.readthedocs.io/en/stable/

In [1]:
import py2neo 
print(py2neo.__version__)

4.3.0


## Connect
Connect to Neo4j with the Graph class.

In [2]:
from py2neo import Graph

graph = Graph()

In [3]:
graph.delete_all()

## Nodes
Create nodes with the Node class. The first argument is the node's label. The remaining arguments are an arbitrary amount of node properties or key-value pairs.

In [4]:
from py2neo import *

In [5]:
nicole = Node("Person", name="Nicole", age=24)
drew = Node("Person", name="Drew", age=20)

mtdew = Node("Drink", name="Mountain Dew", calories=9000)
cokezero = Node("Drink", name="Coke Zero", calories=0)

coke = Node("Manufacturer", name="Coca Cola")
pepsi = Node("Manufacturer", name="Pepsi")

graph.create(nicole | drew | mtdew | cokezero | coke | pepsi)

In [9]:
drew.update(city = 'NY', country = 'US')

In [10]:
graph.push(drew)

ClientError: SyntaxError: The old parameter syntax `{param}` is no longer supported. Please use `$param` instead (line 1, column 25 (offset: 24))
"MATCH (_) WHERE id(_) = {x}"
                         ^

We have only one Nicole!

In [7]:
graph.merge(Node("Person", name="Nicole", age=24))

ValueError: Primary label and primary key are required for MERGE operation

In [8]:
from scripts.vis import draw

options = {"Person": "name", "Drink": "name", "Manufacturer": "name"}
draw(graph, options)

## Relationships
Create relationships between nodes with the Relationship class.

In [ ]:
from py2neo import Relationship

rel_nicole_cokezero = Relationship(nicole, "LIKES", cokezero)
rel_nicole_mtdew = Relationship(nicole, "LIKES", mtdew)
rel_drew_mtdew = Relationship(drew, "LIKES", mtdew)
rel_coke_cokezero = Relationship(coke, "MAKES", cokezero)
rel_pepsi_mtdew = Relationship(pepsi, "MAKES", mtdew)

graph.create(rel_nicole_cokezero | rel_nicole_mtdew | rel_drew_mtdew |  rel_coke_cokezero | rel_pepsi_mtdew)

draw(graph, options)

## Relationships with atributes

In [ ]:
rel_drew_pepsi = Relationship(drew, "WORKS", pepsi, years = 5)

graph.create(rel_drew_pepsi)

## Cypher
Retrieve Cypher query results with Graph.cypher.execute.

In [ ]:
query = """
MATCH (person:Person)-[:LIKES]->(drink:Drink)
RETURN person.name AS name, person.city as City, drink.name AS drink
"""

data = graph.run(query)

for d in data:
    print(d)

## Parameterized Cypher
Pass parameters to Cypher queries by passing additional key-value arguments to Graph.cypher.execute. Parameters in Cypher are named and are wrapped in curly braces.

In [ ]:
query = """
MATCH (p:Person)-[:LIKES]->(drink:Drink)
WHERE p.name = {name}
RETURN p.name AS name, AVG(drink.calories) AS avg_calories
"""

data = graph.run(query, name="Nicole")

for d in data:
    print(d)

## ipython-cypher
ipython-cypher exposes %cypher magic in Jupyter. Install ipython-cypher with pip install ipython-cypher.

In [ ]:
%load_ext cypher

## Cypher
%cypher is intended for one-line Cypher queries and %%cypher is intended for multi-line Cypher queries. Placing %%cypher above a Cypher query will display that query's results.

In [ ]:
%%cypher
MATCH (person:Person)-[:LIKES]->(drink:Drink)
RETURN person.name, drink.name, drink.calories

## Pandas Data Frames
Cypher query results can be coerced to pandas data frames with the get_dataframe method. To assign Cypher query results to a variable, you need to use **%cypher** and separate lines with \ 

You'll first need to install pandas with pip install pandas.

In [ ]:
results = %cypher MATCH (person:Person)-[:LIKES]->(drink:Drink) \
                  RETURN person.name AS name, drink.name AS drink
    
df = results.get_dataframe()

df

## NetworkX Graphs
Cypher query results can be coerced to NetworkX MultiDiGraphs, graphs that permit multiple edges between nodes, with the get_graph method

In [ ]:
import networkx as nx
%matplotlib inline

results = %cypher MATCH p = (:Person)-[:LIKES]->(:Drink) RETURN p

g = results.get_graph()

nx.draw(g)

In [ ]:
g.nodes(data=True)

In [ ]:
nx.degree(g)

## igraph
Cypher query results can be imported into igraph with py2neo. You'll need to install igraph with pip install python-igraph. Query results should be returned as edgelists, as igraph has a method for building an igraph object from a list of tuples representing edges between nodes.

In [ ]:
from py2neo import Graph as PGraph
from igraph import Graph as IGraph

neo4j = PGraph()

query = """
MATCH (person:Person)-[:LIKES]->(drink:Drink)
RETURN person.name AS source, drink.name AS target
"""

data = neo4j.run(query)
tups = []

for d in data:
    tups.append((d["source"], d["target"]))

In [ ]:
from igraph import Graph

In [ ]:
ig = IGraph.TupleList(tups)

ig

In [ ]:
best = ig.vs.select(_degree = ig.maxdegree())["name"]
best